## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,68.00,66.00,91,41,1.97,light rain
1,1,Atuona,PF,-9.80,-139.03,78.13,78.13,78,5,17.05,clear sky
2,2,Mataura,NZ,-46.19,168.86,60.01,60.01,72,50,0.74,light rain
3,3,Senador Pompeu,BR,-5.59,-39.37,71.91,71.91,82,14,3.40,few clouds
4,4,Cherskiy,RU,68.75,161.30,39.36,39.36,93,100,8.81,light rain


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = pd.to_numeric(input("Please enter desired minimum temperature:"))
max_temp = pd.to_numeric(input("Please enter desired maximum temperature:"))

Please enter desired minimum temperature:42.0
Please enter desired maximum temperature:69.6


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df["Max Temp"] = city_data_df["Max Temp"].astype(float)
city_data_df["Min Temp"] = city_data_df["Min Temp"].astype(float)
temp_cities = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Min Temp"] >= min_temp)]
temp_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,68.00,66.00,91,41,1.97,light rain
2,2,Mataura,NZ,-46.19,168.86,60.01,60.01,72,50,0.74,light rain
6,6,Thompson,CA,55.74,-97.86,53.60,53.60,100,90,6.93,light rain
7,7,Fortuna,US,40.60,-124.16,57.99,55.40,93,90,6.93,overcast clouds
8,8,Cape Town,ZA,-33.93,18.42,57.99,55.00,93,20,11.41,few clouds
...,...,...,...,...,...,...,...,...,...,...,...
672,672,Huarmey,PE,-10.07,-78.15,59.97,59.97,86,12,8.63,few clouds
677,677,Tarko-Sale,RU,64.92,77.78,60.01,60.01,85,46,3.51,scattered clouds
678,678,Colesberg,ZA,-30.72,25.10,45.90,45.90,56,74,5.39,broken clouds
686,686,Guapore,BR,-28.85,-51.89,54.00,53.01,69,0,4.54,clear sky


In [17]:
# 4a. Determine if there are any empty rows.
temp_cities.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Min Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = temp_cities.dropna()

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,68.00,light rain,42.60,-73.97,
2,Mataura,NZ,60.01,light rain,-46.19,168.86,
6,Thompson,CA,53.60,light rain,55.74,-97.86,
7,Fortuna,US,57.99,overcast clouds,40.60,-124.16,
8,Cape Town,ZA,57.99,few clouds,-33.93,18.42,
11,Bluff,NZ,60.01,broken clouds,-46.60,168.33,
13,Dikson,RU,46.13,clear sky,73.51,80.55,
14,Ovalle,CL,50.88,overcast clouds,-30.60,-71.20,
16,Khatanga,RU,66.34,overcast clouds,71.97,102.50,
17,Margate,GB,60.01,overcast clouds,51.38,1.39,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    hot_lat = row["Lat"]
    hot_lng = row["Lng"]
    
    params["location"] = f"{hot_lat},{hot_lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))